In [1]:
#Print start time
print(date())

[1] "Fri Nov  6 16:10:55 2015"


In [2]:
# Set the seed of R‘s random number generator
set.seed(1234567890)

library("neuralnet")

Loading required package: grid
Loading required package: MASS


In [3]:
dataset <- read.csv("merge_all.csv", header=TRUE, stringsAsFactors=FALSE)
# Split the dataset into train and validation sets in 70:30 ratio
index <- round(0.7*nrow(dataset))
trainset <- dataset[1:index, ]
validset <- dataset[(index+1):nrow(dataset), ]

print(nrow(dataset))

[1] 609064


In [4]:
# Build the neural network (NN)
eventNet <- neuralnet(HandStart + FirstDigitTouch + BothStartLoadPhase +
                        LiftOff + Replace + BothReleased
                      ~ Fp1 + Fp2 + F7 + F3 + Fz + F4 + F8, 
                      trainset, hidden = 4, lifesign = "minimal", 
                      linear.output = FALSE, threshold = 0.1)

summary(eventNet)

hidden: 4    thresh: 0.1    rep: 1/1    steps:    6258	error: 31376.84186	time: 1.15 hours


                    Length  Class      Mode    
call                      7 -none-     call    
response            2558070 -none-     numeric 
covariate           2984415 -none-     numeric 
model.list                2 -none-     list    
err.fct                   1 -none-     function
act.fct                   1 -none-     function
linear.output             1 -none-     logical 
data                     39 data.frame list    
net.result                1 -none-     list    
weights                   1 -none-     list    
startweights              1 -none-     list    
generalized.weights       1 -none-     list    
result.matrix            65 -none-     numeric 

In [5]:
# Test the resulting output
# 1. Subset of only the predictor variables
# 2. Apply model on validation set
# 3. Compare actual vs predicted results
temp_valid <- subset(validset, select = c("Fp1","Fp2","F7","F3","Fz","F4","F8"))
eventNet.results <- compute(eventNet, temp_valid)

# Multi class classifier
prediction.results <- data.frame(prediction = eventNet.results$net.result)
names(prediction.results)[1] <- "prediction.HandStart"
names(prediction.results)[2] <- "prediction.FirstDigitTouch"
names(prediction.results)[3] <- "prediction.BothStartLoadPhase"
names(prediction.results)[4] <- "prediction.LiftOff"
names(prediction.results)[5] <- "prediction.Replace"
names(prediction.results)[6] <- "prediction.BothReleased"
head(prediction.results)

,prediction.HandStart,prediction.FirstDigitTouch,prediction.BothStartLoadPhase,prediction.LiftOff,prediction.Replace,prediction.BothReleased
426346,0.02878042574,0.02009173095,0.02290924053,0.02670481058,0.009573084229,0.01452794915
426347,0.02878042574,0.02009173095,0.02290924053,0.02670481058,0.009573084229,0.01452794915
426348,0.02878042574,0.02009173095,0.02290924053,0.02670481058,0.009573084229,0.01452794915
426349,0.02878042574,0.02009173095,0.02290924053,0.02670481058,0.009573084229,0.01452794915
426350,0.02878042574,0.02009173095,0.02290924053,0.02670481058,0.009573084229,0.01452794915
426351,0.02878042574,0.02009173095,0.02290924053,0.02670481058,0.009573084229,0.01452794915


In [6]:
# Round the prediction results
prediction.results.round <- round(prediction.results)  
head(prediction.results.round)

,prediction.HandStart,prediction.FirstDigitTouch,prediction.BothStartLoadPhase,prediction.LiftOff,prediction.Replace,prediction.BothReleased
426346,0,0,0,0,0,0
426347,0,0,0,0,0,0
426348,0,0,0,0,0,0
426349,0,0,0,0,0,0
426350,0,0,0,0,0,0
426351,0,0,0,0,0,0


In [7]:
# Put multiple binary output to categorical output
# Below function assumes the events are mutually exclusive
maxidx <- function(arr) {
  idx <- 0
  if (max(arr) == 0) idx <- 7 
  else idx <- which(arr == max(arr))
  return (idx)
}

prediction.idx <- apply(prediction.results.round, c(1), maxidx)
prediction <- c('handStart', 'firstDigitTouch', 'bothStartLoadPhase',
                'liftOff', 'replace', 'bothReleased', 'none')[prediction.idx]
head(prediction)

[1] "none" "none" "none" "none" "none" "none"

In [8]:
# Issue: apply function did not work on the actual as it returns a list instead of numeric vector
# Suspected: http://www.r-bloggers.com/type-conversion-and-you-or-and-r/
# Workaround:
# Below approach assumes the events are mutually exclusive
actual.results <- data.frame(validset[,2:7])
names(actual.results)[1] <- "handStart"
names(actual.results)[2] <- "firstDigitTouch"
names(actual.results)[3] <- "bothStartLoadPhase"
names(actual.results)[4] <- "liftOff"
names(actual.results)[5] <- "replace"
names(actual.results)[6] <- "bothReleased"
actual.results$none <- (validset$HandStart == 0) & (validset$FirstDigitTouch == 0) &
                        (validset$BothStartLoadPhase == 0) & (validset$LiftOff == 0) &
                        (validset$Replace == 0) & (validset$BothReleased == 0)
head(actual.results)
actual <- cbind(names(actual.results)[max.col(actual.results == 1)])
head(actual)

,handStart,firstDigitTouch,bothStartLoadPhase,liftOff,replace,bothReleased,none
426346,0,0,0,0,0,0,TRUE
426347,0,0,0,0,0,0,TRUE
426348,0,0,0,0,0,0,TRUE
426349,0,0,0,0,0,0,TRUE
426350,0,0,0,0,0,0,TRUE
426351,0,0,0,0,0,0,TRUE


none
none
none
none
none
none


In [9]:
table(prediction, actual)

          actual
prediction bothReleased bothStartLoadPhase firstDigitTouch handStart liftOff
      none         3017               1824            2179      3450    2941
          actual
prediction   none replace
      none 166282    3026

In [10]:
#Print end time
print(date())

[1] "Fri Nov  6 17:22:13 2015"
